In [4]:
!pip install ipywidgets --upgrade
!pip install jupyter --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 9.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.7 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.9 MB/s  0:00:00eta 0:00:01
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 46.4 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51/51 [jupyter]9/51 [notebook]b]ver]nt]


In [2]:
!pip install gradio

  Using cached gradio-5.49.1-py3-none-any.whl.metadata (16 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached ffmpy-0.6.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached gradio_client-1.13.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached pydantic-2.11.10-py3-none-any.whl.metadata (68 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.7-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached annotated_doc-0.0.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.met

In [6]:
import gradio as gr
import pandas as pd
import joblib
import mediapipe as mp
import cv2
import time
import numpy as np

In [7]:
classification_model = joblib.load('./SVM_Models/best_label_classification_model.pkl')

In [8]:
classification_model

,steps,"[('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False


In [9]:
# Mediapipe setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.8, min_tracking_confidence=0.7)

I0000 00:00:1762033460.563975  170910 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1762033460.569739  176274 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762033460.574025  176274 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [10]:
def encode_fingers(hand_landmarks, hand_type):
    tips = [mp_hands.HandLandmark.THUMB_TIP,
            mp_hands.HandLandmark.INDEX_FINGER_TIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
            mp_hands.HandLandmark.RING_FINGER_TIP,
            mp_hands.HandLandmark.PINKY_TIP]
    dips = [mp_hands.HandLandmark.THUMB_IP,
            mp_hands.HandLandmark.INDEX_FINGER_DIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
            mp_hands.HandLandmark.RING_FINGER_DIP,
            mp_hands.HandLandmark.PINKY_DIP]

    finger_status = []
    for i, (tip_id, dip_id) in enumerate(zip(tips, dips)):
        tip = hand_landmarks.landmark[tip_id]
        dip = hand_landmarks.landmark[dip_id]
        if i == 0:
            is_open = 1 if (tip.x < dip.x if hand_type == "Right" else tip.x > dip.x) else 0
        else:
            is_open = 1 if tip.y < dip.y else 0
        finger_status.append(is_open)
    return finger_status

In [11]:
# Video processing function
def process_frame(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    features = [0]*10
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            hand_type = results.multi_handedness[hand_idx].classification[0].label
            fingers = encode_fingers(hand_landmarks, hand_type)
            if hand_type == "Left": features[:5] = fingers
            elif hand_type == "Right": features[5:] = fingers
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    return frame, features

In [12]:
# Main prediction loop for Gradio
def run_live_feed():
    cap = cv2.VideoCapture(0)
    latency_list = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        start = time.time()
        frame = cv2.flip(frame, 1)
        frame, features = process_frame(frame)
        if any(features):
            input_df = pd.DataFrame([features], columns=classification_model.feature_names_in_)
            prediction = classification_model.predict(input_df)[0]
        else:
            prediction = "No Hand"
        latency = (time.time() - start)*1000
        latency_list.append(latency)
        cv2.putText(frame, f"Gesture: {prediction}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.putText(frame, f"Latency: {latency:.1f} ms", (10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        yield frame  # stream frame to Gradio video component
    cap.release()

In [13]:
# Simple UFID login system (simulate)
user_database = {}

def register_ufid(ufid):
    if ufid in user_database:
        return f"UFID {ufid} already registered! Please log in.", gr.update(interactive=False)
    else:
        user_database[ufid] = True
        return f"UFID {ufid} registered successfully! You can now start.", gr.update(interactive=True)


In [14]:
def login_ufid(ufid):
    if ufid in user_database:
        return f"Welcome back, UFID {ufid}! Click Start to begin.", gr.update(interactive=True)
    else:
        return "UFID not found! Please register first.", gr.update(interactive=False)


In [15]:
with gr.Blocks() as demo:
    gr.Markdown("## 🖐️ Hand Gesture Detection System (UF Project)")

    with gr.Tab("Login / Register"):
        ufid = gr.Textbox(label="Enter your UFID:")
        register_btn = gr.Button("Register")
        login_btn = gr.Button("Login")
        status = gr.Textbox(label="Status", interactive=False)
        start_btn = gr.Button("Start Detection", interactive=False)
        register_btn.click(register_ufid, inputs=ufid, outputs=[status, start_btn])
        login_btn.click(login_ufid, inputs=ufid, outputs=[status, start_btn])

    with gr.Tab("Live Detection"):
        live_feed = gr.Image(label="Camera Feed", streaming=True)
        emergency_btn = gr.Button("🚨 Emergency Stop")
        feedback = gr.Textbox(label="Feedback", placeholder="Write your feedback here...")
        start_btn.click(run_live_feed, outputs=live_feed)
        emergency_btn.click(lambda: "System stopped due to emergency!", outputs=status)


In [ ]:
demo.launch(share = True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


W0000 00:00:1762033609.661027  176277 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
